In [2]:
import pandas as pd
import re
import nltk
import numpy as np

from nltk.corpus import sentiwordnet as swn

In [7]:
# 처음에 nltk 가 import 안될 경우는 패키지가 제대로 다운이 되지 않은 것 때문이므로 아래 명령어로 다운로드
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
상폐기업뉴스1 = pd.read_csv('상폐기업번역파일.csv')
상폐기업뉴스1.drop('뉴스기사본문전처리', axis=1, inplace=True)

상폐기업뉴스2 = pd.read_csv('상폐기업번역파일추가.csv')
상폐기업뉴스2.drop('뉴스기사본문전처리', axis=1, inplace=True)

상폐기업뉴스 = pd.concat([상폐기업뉴스1, 상폐기업뉴스2], axis=0)
상폐기업뉴스.reset_index(inplace=True, drop=True)
상폐기업뉴스.drop('기사발행일', inplace=True)

상폐기업뉴스['기사본문번역'] = [re.sub('[가-힣]', '', s) for s in 상폐기업뉴스['기사본문번역']]

상폐기업뉴스['기사발행일'] = 상폐기업뉴스['기사발행일'].apply(lambda x: x[:4])


,기업,기사발행일,기사제목,기사본문번역
0,씨제이이엔엠,2011,"씨제이이엔엠, 유형자산 취득 후 처분 예정",CJ ENM announced that it will acquire tangible...
1,씨제이이엔엠,2011,시초가 소폭 하락 예상 - 코스닥 동시호가,The Japanese KOSDAQ market is expected to star...
2,안랩,2011,"안랩, `V3ㆍ네트워크 보안장비 연동` 특허 획득",Ahn Cheol SooThe institute announced that the ...
3,안랩,2011,"한컴과 안랩, 10월은 특별한 달","In the wake of the month, Korea's leading soft..."
4,포스코아이씨티,2011,"포스코아이씨티, POSCO IT Outsourcing SLA 계약금액 증가",POSCO ICT announced that it would revise and d...
...,...,...,...,...
6873,에이치엘비,2020,"""이래도 불법 공매도가 아니라고?""...에이치엘비 주주들 뿔났다",HLB shareholders are raising suspicions that i...
6874,에이치엘비,2020,"리보세라닙 병용 식도암 임상, 방사선 병행시 3명중 1명 `완전관해`",HLB announced that it once again secured stati...
6875,에이치엘비,2020,"넥스트사이언스 관계사 나노젠 ""내년 5월 코로나백신 양산 계획""","Next Science announced that Nanogen, the only ..."
6876,에이치엘비,2020,넥스트사이언스 관계사 나노젠 “코로나 백신 내년 5월 양산 계획”,"Nanocobox, made from photoNextScience recombin..."


In [10]:
상폐기업뉴스['기사본문번역토큰'] = 상폐기업뉴스['기사본문번역'].apply(lambda x: nltk.word_tokenize(x))
상폐기업뉴스['기사본문번역토큰'] = 상폐기업뉴스['기사본문번역토큰'].apply(lambda x: nltk.pos_tag(x))

In [11]:
def word_sentiment_calculator(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))

In [13]:
def sentence_sentiment_calculator(sent):
    tokens =  nltk.word_tokenize(sent)
    pos_tags = nltk.pos_tag(tokens)
    
    pos_score = 0
    neg_score = 0
    for word, tag in pos_tags:
        pos_score += word_sentiment_calculator(word, tag)[0]
        neg_score += word_sentiment_calculator(word, tag)[1]
    return (pos_score, neg_score)

In [15]:
## 긍정일경우 0 부정일경우 1
predicted = []

for i in range(len(상폐기업뉴스)):
    scores = sentence_sentiment_calculator(상폐기업뉴스.기사본문번역[i])
    
    if scores[0] >= scores[1]:
        predicted.append(0)
    else:
        predicted.append(1)

상폐기업뉴스['긍정부정분류'] = predicted

In [18]:
전체기사수 = pd.DataFrame(상폐기업뉴스.groupby(['기업']).count()['긍정부정분류'])
전체기사수.reset_index(drop = False, inplace = True)
전체기사수.columns = ['기업', '전체기사수']

부정기사수 = pd.DataFrame(상폐기업뉴스[상폐기업뉴스.긍정부정분류==1].groupby(['기업']).count()['긍정부정분류'])
부정기사수.reset_index(drop = False, inplace = True)
부정기사수.columns = ['기업', '부정기사수']

상폐기업 = pd.merge(전체기사수, 부정기사수, on=['기업'], how='left')
상폐기업['부정기사비율'] = (상폐기업['부정기사수'] / 상폐기업['전체기사수']) * 100
상폐기업.fillna(0, inplace=True)
상폐기업.drop(['전체기사수', '부정기사수'], axis=1, inplace=True)
상폐기업.columns = ['회사명', '부정기사비율']
상폐기업

,회사명,회계년도,부정기사비율
0,경창산업,2011,8.064516
1,고영테크놀러지,2009,4.255319
2,광림,2014,25.454545
3,금화피에스시,2011,28.169014
4,나스미디어,2015,11.594203
...,...,...,...
119,하이록코리아,2010,1.408451
120,한국정보통신,2013,9.090909
121,해성산업,2011,14.285714
122,홈캐스트,2014,23.913043


In [24]:
상폐기업재무비율 = pd.read_csv('../Step1-2_재무비율_모델링/재무비율.csv')
상폐기업재무비율 = 상폐기업재무비율[상폐기업재무비율.부실기업여부==1]
상폐기업_감성분석 = pd.merge(상폐기업재무비율, 상폐기업, how='left', on=['회사명'])
상폐기업_감성분석.fillna(0, inplace=True)
상폐기업_감성분석.to_csv('상폐기업_감성분석_데이터.csv', index=None, encoding='utf-8-sig')
